In [75]:
from Classes.Person import Person
import pickle
import pandas as pd

In [76]:
honest = pickle.load(open("batch_run_honest_1","rb"))
strategic = pickle.load(open("batch_run_strategic_1","rb"))

In [77]:
honest.head()

,Run,Polls,Votes,n_voters,n_candidates,voter_type,maxpolls,loyalty,width,height
0,1000,[{<Classes.Person.Candidate object at 0x000001...,{'cand0': <bound method Candidate.getVotes of ...,1000,3,Honest,6,30,2,2
1,1000,[{<Classes.Person.Candidate object at 0x000001...,{'cand0': <bound method Candidate.getVotes of ...,1000,3,Honest,6,30,2,2
2,1000,[{<Classes.Person.Candidate object at 0x000001...,{'cand0': <bound method Candidate.getVotes of ...,1000,3,Honest,6,30,2,2
3,1000,[{<Classes.Person.Candidate object at 0x000001...,{'cand0': <bound method Candidate.getVotes of ...,1000,3,Honest,6,30,2,2
4,1000,[{<Classes.Person.Candidate object at 0x000001...,{'cand0': <bound method Candidate.getVotes of ...,1000,3,Honest,6,30,2,2


In [78]:
def convertRuns(_data):
    runs = pd.DataFrame(columns=['run_id', 'cand_unique_id', 'cand_votes', 'cand_position'])
    for i, run in _data.iterrows():
        for cand in run[1][0]:
            runs = runs.append({'run_id':i,'cand_unique_id':cand.unique_id,'cand_votes':run[1][0].get(cand),'cand_position':cand.position},ignore_index=True)
    return runs

In [80]:
honest_runs = convertRuns(honest) 
honest_runs.head()

,run_id,cand_unique_id,cand_votes,cand_position
0,0,0,254,"[1.7927997466683059, 0.7689119541395559]"
1,0,1,157,"[1.2378404310031115, 0.12756330309259423]"
2,0,2,589,"[0.9918050245511618, 0.82057497560806]"
3,1,0,426,"[1.5036175201821793, 1.4161317682684034]"
4,1,1,235,"[0.561317853004121, 1.7943473285402216]"


In [81]:
strategic_runs = convertRuns(strategic) 
strategic_runs.head()

,run_id,cand_unique_id,cand_votes,cand_position
0,0,0,338,"[1.7337125266426066, 0.6822431371780469]"
1,0,1,235,"[0.8887795688981128, 0.29222290811091756]"
2,0,2,427,"[0.415227857735595, 1.2057487665921145]"
3,1,0,268,"[0.6095514887623634, 0.3085336440127491]"
4,1,1,629,"[0.6949512236550253, 0.9431161343433059]"
